In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-project
Azure region: eastus
Subscription id: fc6c6e9e-bb6e-4590-ab2f-0933591ec61b
Resource group: udacity-vishwa


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use  in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "Standard_D2_V2"

cpu_cluster_name = 'udacity_cluster_project'

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())



Creating a new compute target...


ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '375', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': 'f77c627f-4bf3-4569-b2bb-4994999fe003', 'x-ms-ratelimit-remaining-subscription-writes': '1199', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-890039e43eda13429cbfd29a8468b29b-98c9d64af1d1224b-00.3dcb715b_', 'x-request-time': '0.361', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'EASTUS:20201002T183134Z:f77c627f-4bf3-4569-b2bb-4994999fe003', 'X-Content-Type-Options': 'nosniff', 'Date': 'Fri, 02 Oct 2020 18:31:34 GMT'}
Content: b'{"error":{"code":"BadArgument","message":"Compute name is invalid. It can include letters, digits and dashes. It must start wi

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling({'C':uniform(0,1),"max_iter":uniform(50,200)})
# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1,delay_evaluation=5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = 'vbrungi',
              compute_target= cpu_cluster, 
              vm_size= vm_size,
              entry_script='train.py', 
              script_params= ['--C',0.5,'--max_iter',100])

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='validation_acc', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
experiment = Experiment(ws, exp)
hyperdrive_run = experiment.submit(hyperdrive_config)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(parameter_values)

joblib.dump(best_run, 'outputs/best_model.joblib')


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = Dataset.Tabular.from_delimited_files(path=web_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    X = x,
    y = y,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run

from azureml.core.experiment import Experiment

experiment = Experiment(ws, "automl_experiment")
run = experiment.submit(config=automl_config, show_output=True)


In [ ]:
# Retrieve and save your best automl model.

best_automl_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_automl_run.get_metrics()
parameter_values = best_automl_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(parameter_values)

joblib.dump(best_run, 'outputs/best_automl_model.joblib')
